In [1]:
import os
import pandas as pd
import requests
from io import StringIO

# Get the current working directory 
current_directory = os.getcwd() 

# Print the current working directory 
print(current_directory)

# Define the directory containing the files 
path_NOAA = current_directory+"\\Data\\NOAA\\"
path_NSRDB = current_directory+"\\Data\\NSRDB\\"
print(path_NOAA, path_NSRDB)

dataframes = []

filename_NOAA = path_NOAA + 'HADISDH_NOAA_Weather_Station_Matches.csv'

weather_stations_df = pd.read_csv(filename_NOAA) #read weather station data into a dataframe

print(weather_stations_df)

for filename_NSRDB in os.listdir(path_NSRDB):
    file_path = os.path.join(path_NSRDB, filename_NSRDB)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            content_df = pd.read_csv(file)
            content_df['file_id'] = int(content_df['file_id'].values[0])
            content_df['region'] = int(content_df['region'].values[0])
            content_df['water'] = int(content_df['water'].values[0])
            # Merge data frames for HADISDH/NSRDB and NOAA data
            result_df = pd.merge(content_df, weather_stations_df, on='file_id', how='left')
            result_df = result_df.drop(['HADISDH_Latitude','HADISDH_Longitude', 'NOAA_Latitude', 'NOAA_Longitude'], axis=1)
            result_df = result_df.drop(['NSRDB_latitude','NSRDB_longitude', 'Distance', 'distance_h_NSRDB'], axis=1)
            result_df['NOAA_file_csv'] = (result_df['WMO_ID'].values[0])[:6]+(result_df['WMO_ID'].values[0])[7:]+".csv"
            result_df['DATE'] = result_df['time']
            NOAA_dataframes = []
            for i in range(19):
                year = 2006 + i # years 2006-2024
                print(str(year)+": File: "+result_df['NOAA_file_csv'].values[0])
                url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/'+ str(year) +'/'+ result_df['NOAA_file_csv'].values[0]
                response = requests.get(url)
                print(url)

                if response.status_code == 200:
                    content = response.text
                    content_df = pd.read_csv(StringIO(content))
                    content_df = content_df.drop(['LATITUDE', 'LONGITUDE', 'ELEVATION'], axis=1)
                    NOAA_dataframes.append(content_df)    
                else:
                    print(f"Failed to retrieve the file. Status code: {response.status_code}")
            combined_df = pd.concat(NOAA_dataframes)
            result2_df = pd.merge(result_df, combined_df, on='DATE', how='left')
            print(result2_df)
            dataframes.append(result2_df)

C:\Users\User
C:\Users\User\Data\NOAA\ C:\Users\User\Data\NSRDB\
     file_id  HADISDH_Latitude  HADISDH_Longitude        WMO_ID  \
0       6501            33.636            -91.756  720175-53919   
1       6541            38.958            -94.371  720306-53879   
2       6640            28.474            -82.454  722014-12818   
3       6668            35.140            -90.236  722054-53959   
4       6678            30.393            -86.467  722069-53853   
..       ...               ...                ...           ...   
101     8184            36.985           -120.111  745046-93242   
102     8186            33.038           -116.916  745056-53120   
103     8247            30.337            -81.513  747820-53860   
104     8248            26.079            -80.162  747830-12849   
105     9858            36.118            -97.091  999999-53927   

     NOAA_Latitude  NOAA_Longitude  Distance  
0           33.636         -91.756      0.00  
1           38.958         -94.371  

In [2]:
combined_all_df = pd.concat(dataframes)

In [3]:
# Write DataFrame to CSV
combined_all_df.to_csv('Daily_Data_All.csv', index=False)

In [4]:
# Write DataFrame to CSV
combined_all_df.to_csv('Daily_Data_All_Index_True.csv', index=True)